In [1]:
import os
from datetime import date
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser #responsible for parsing the output of a model into a more accessible format
from langchain.globals import set_debug
from dotenv import load_dotenv

set_debug(True)

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv('ANTHROPIC_API_KEY')

#initialize model
chat_model = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0
)

#chat model uses chat messages as input/output. So resulting message: AIMessage(content="...") looks like that. 
chat_model.invoke([
    HumanMessage("Tell me a joke about bears!")
])

#construct a prompt template consisting of templates for a SystemMessage and a HumanMessage using from_messages. 
#SystemMessages are meta-instructions that are not part of the current conversation, but purely guide input.
#
joke_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class comedian."),
    ("human", "Tell me a joke about {topic}") #The prompt template contains {topic} in curly braces. This denotes a required parameter named "topic"
])

#You can compose Runnables into “chains” using the pipe (|) operator where you .invoke() the next step with the output of the previous one.

chain = joke_prompt | chat_model
joke_prompt.invoke({"topic": "beets"}) #You invoke the prompt template with a dict with a key named "topic" and a value "beets"
chain.invoke({"topic": "beets"})

"""If you want to work with the raw string output of the of the message:
LangChain has a component called an Output Parser, which, as the name implies, is responsible for parsing the output of a model into a more accessible format. 
Since composed chains are also Runnable, you can again use the pipe operator:
"""
str_chain = chain | StrOutputParser()
str_chain.invoke({"topic": "beets"})

#You can iterate over the output using for ... in syntax. Try it with the str_chain you just declared:
for chunk in str_chain.stream({"topic": "beets"}):
    print(chunk, end="|")

chat_model = ChatAnthropic(model_name="claude-3-sonnet-20240229")

prompt = ChatPromptTemplate.from_messages([
    ("system", 'You know that the current date is "{current_date}".'),
    ("human", "{question}")
])

chain = prompt | chat_model | StrOutputParser()

chain.invoke({
    "question": "What is the current date?",
    "current_date": date.today()
})

#Simple RAG example:

chat_model = ChatAnthropic(model_name="claude-3-sonnet-20240229")

SOURCE = """
Old Ship Saloon 2023 quarterly revenue numbers:
Q1: $174782.38
Q2: $467372.38
Q3: $474773.38
Q4: $389289.23
"""

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", 'You are a helpful assistant. Use the following context when responding:\n\n{context}.'),
    ("human", "{question}")
])

rag_chain = rag_prompt | chat_model | StrOutputParser()

rag_chain.invoke({
    "question": "What was the Old Ship Saloon's total revenue in Q1 2023?",
    "context": SOURCE
})


[llm/start] [llm:ChatAnthropic] Entering LLM run with input:
{
  "prompts": [
    "Human: Tell me a joke about bears!"
  ]
}
[llm/end] [llm:ChatAnthropic] [1.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here's a bear joke for you:\n\nWhy did the bear dissolve in water?\nBecause it was a polar bear!",
        "generation_info": null,
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Here's a bear joke for you:\n\nWhy did the bear dissolve in water?\nBecause it was a polar bear!",
            "response_metadata": {
              "id": "msg_01AXcbRt93GHwDWamTHUjAnC",
              "model": "claude-3-sonnet-20240229",
              "stop_reason": "end_turn",
              "stop_sequence": null,
              "usage": {
     

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of requests has exceeded your per-minute rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please try again later or contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}

In [2]:
#Simple RAG example:

chat_model = ChatAnthropic(model_name="claude-3-sonnet-20240229")

SOURCE = """
Old Ship Saloon 2023 quarterly revenue numbers:
Q1: $174782.38
Q2: $467372.38
Q3: $474773.38
Q4: $389289.23
"""

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", 'You are a helpful assistant. Use the following context when responding:\n\n{context}.'),
    ("human", "{question}")
])

rag_chain = rag_prompt | chat_model | StrOutputParser()

rag_chain.invoke({
    "question": "What was the Old Ship Saloon's total revenue in Q1 2023?",
    "context": SOURCE
})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What was the Old Ship Saloon's total revenue in Q1 2023?",
  "context": "\nOld Ship Saloon 2023 quarterly revenue numbers:\nQ1: $174782.38\nQ2: $467372.38\nQ3: $474773.38\nQ4: $389289.23\n"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "What was the Old Ship Saloon's total revenue in Q1 2023?",
  "context": "\nOld Ship Saloon 2023 quarterly revenue numbers:\nQ1: $174782.38\nQ2: $467372.38\nQ3: $474773.38\nQ4: $389289.23\n"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatAnthropic] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful assistant. Use the following context when responding:\n\n\nOld Ship Saloon 2023 quarterly revenue numbers:\nQ1: $174782.38\nQ2: $467372.38\nQ3: $474773.38\nQ4: $389289.2

"According to the provided context, the Old Ship Saloon's revenue for Q1 2023 was $174,782.38."